In [1]:
# Geospatial data processing
import geopandas as gpd
from shapely import Point
from tqdm.auto import tqdm # Import a biblioteca de barra de progresso
import warnings

# Mapping and visualization
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.lines import Line2D

# The star of the show: city2graph for transportation network analysis
import city2graph as c2g
import city2graph.utils as c2g_utils

# Configure matplotlib for publication-quality visualizations
plt.rcParams['figure.figsize'] = (15, 12)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11
plt.style.use('ggplot')
#plt.style.use('default')  # Clean default style instead of ggplot

print("All dependencies loaded successfully!")
print(f"city2graph version: {c2g.__version__ if hasattr(c2g, '__version__') else 'development'}")

All dependencies loaded successfully!
city2graph version: 0.1.6


In [2]:
# Suprimir avisos de operações do Shapely (ex: .difference)
warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)

# Defina o CRS Projetado (ex: SIRGAS 2000 / UTM zone 23S para Teresina)
CRS_PROJETADO = 31983

## Carregando os dados

In [3]:
# --- 2. CARREGAR DADOS DE ENTRADA ---

print("Carregando arquivos de entrada...")
# 2.1. Carregar a Rede de Ruas (já processada)
# (Saída da "ETAPA 4" do script anterior)
try:
    edges_gdf = gpd.read_parquet("./dados/overturemaps/finais/grafo_arestas.parquet")
    if edges_gdf.crs.to_epsg() != CRS_PROJETADO:
        print(f"Convertendo CRS das arestas para {CRS_PROJETADO}...")
        edges_gdf = edges_gdf.to_crs(CRS_PROJETADO)
except Exception as e:
    print(f"Erro ao carregar 'grafo_arestas_processado.gpkg': {e}")
    # (Adicione aqui a lógica para gerar o edges_gdf se ele não existir)
    

# 2.2. Carregar o GeoDataFrame das 75 UBS
# (Este arquivo deve ter colunas que você possa usar, 
#  ex: 'cnes', 'nome_unidade' e, claro, 'geometry')
try:
    ubs_gdf = gpd.read_file("./dados/semplam/ubs_zu.geojson")
    if ubs_gdf.crs.to_epsg() != CRS_PROJETADO:
        print(f"Convertendo CRS das UBS para {CRS_PROJETADO}...")
        ubs_gdf = ubs_gdf.to_crs(CRS_PROJETADO)
except Exception as e:
    print(f"Erro ao carregar 'unidades_de_saude.gpkg': {e}")
    # (Saia do script se as UBS não puderem ser carregadas)
    exit()

print(f"Rede carregada: {len(edges_gdf)} arestas.")
print(f"Unidades Básica de Saúde carregadas: {len(ubs_gdf)} pontos.")


Carregando arquivos de entrada...
Convertendo CRS das UBS para 31983...
Rede carregada: 43898 arestas.
Unidades Básica de Saúde carregadas: 75 pontos.


# 5 Cálculo das Isocronas: 0-10, 10-20 e 20-30min

In [10]:
# --- 3. PROCESSAMENTO EM LOTE ---

# Lista para armazenar nossos resultados (geometrias e atributos)
aneis_de_isocronas = []

print(f"\nIniciando geração de isócronas para as {len(ubs_gdf)} unidades...")
# tqdm() envolve o iterador para criar uma barra de progresso
for ubs in tqdm(ubs_gdf.itertuples(), total=len(ubs_gdf)):
    
    # Pegar o ponto de origem (geometria) da UBS atual
    ponto_origem = ubs.geometry
    
    # Pegar um ID único da UBS. 
    # Estou usando o Index (ID da linha), mas você pode usar 
    # ubs.cnes, ubs.nome_unidade, ou qualquer coluna única.
    id_origem = ubs.cnes
    
    try:
        # --- 3.1. Calcular as 3 isócronas cumulativas ---
        poly_10_gdf = c2g_utils.create_isochrone(
            graph=edges_gdf,
            center_point=ponto_origem,
            distance=10, 
            edge_attr='travel_time_min'
        )
        
        poly_20_gdf = c2g_utils.create_isochrone(
            graph=edges_gdf,
            center_point=ponto_origem,
            distance=20, 
            edge_attr='travel_time_min'
        )
        
        poly_30_gdf = c2g_utils.create_isochrone(
            graph=edges_gdf,
            center_point=ponto_origem,
            distance=30, 
            edge_attr='travel_time_min'
        )

        # --- 3.2. Extrair geometrias e tratar falhas ---
        # Se a isócrona não puder ser gerada (ex: ponto isolado),
        # a função create_isochrone retorna um GDF vazio.
        
        if not poly_10_gdf.empty:
            poly_10_cumulativo = poly_10_gdf.geometry.iloc[0]
        else:
            # Pula esta UBS se não conseguir gerar a isócrona de 5 min
            print(f"Aviso: Não foi possível gerar isócrona de 10min para ID {id_origem}. Pulando.")
            continue
            
        if not poly_20_gdf.empty:
            poly_20_cumulativo = poly_20_gdf.geometry.iloc[0]
        else:
            poly_20_cumulativo = poly_20_cumulativo # Evita erro no .difference

        if not poly_30_gdf.empty:
            poly_30_cumulativo = poly_30_gdf.geometry.iloc[0]
        else:
            poly_30_cumulativo = poly_30_cumulativo # Evita erro no .difference

        # --- 3.3. Gerar as zonas "Donut" (Anéis Discretos) ---
        zona_0_10_min = poly_10_cumulativo
        zona_10_20_min = poly_20_cumulativo.difference(poly_10_cumulativo)
        zona_20_30_min = poly_30_cumulativo.difference(poly_20_cumulativo)

        # --- 3.4. Adicionar os resultados à lista ---
        # Adicionamos um dicionário para cada anel
        
        aneis_de_isocronas.append({
            'id_origem': id_origem,
            'zona_tempo': '0-10 min',
            'geometry': zona_0_10_min
        })
        
        aneis_de_isocronas.append({
            'id_origem': id_origem,
            'zona_tempo': '10-20 min',
            'geometry': zona_10_20_min
        })
        
        aneis_de_isocronas.append({
            'id_origem': id_origem,
            'zona_tempo': '20-30 min',
            'geometry': zona_20_30_min
        })

    except Exception as e:
        print(f"Erro inesperado ao processar ID {id_origem}: {e}")

# --- 4. FINALIZAÇÃO ---

print("\nProcessamento em lote concluído.")
print(f"Total de {len(aneis_de_isocronas)} geometrias de anel geradas.")


Iniciando geração de isócronas para as 75 unidades...


100%|██████████| 75/75 [07:18<00:00,  5.85s/it]


Processamento em lote concluído.
Total de 225 geometrias de anel geradas.


In [11]:
# 4.1. Converter a lista de resultados em um GeoDataFrame
print("Convertendo resultados para GeoDataFrame...")
gdf_aneis_final = gpd.GeoDataFrame(
    aneis_de_isocronas,
    crs=CRS_PROJETADO
)

# 4.2. (Opcional) Juntar atributos das UBS (ex: nome)
# Se você usou o Index como 'id_origem', podemos facilmente
# juntar os dados originais das UBS.
if 'id_origem' in gdf_aneis_final.columns:
    # Selecionar colunas de atributos (exceto geometria)
    atributos_ubs = ubs_gdf.drop(columns='geometry') 
    
    gdf_aneis_final = gdf_aneis_final.merge(
        atributos_ubs,
        left_on='id_origem',
        #right_index=True,
        right_on='cnes',
        how='left'
    )
    print("Atributos das UBS (nome, cnes, etc.) juntados ao resultado.")

Convertendo resultados para GeoDataFrame...
Atributos das UBS (nome, cnes, etc.) juntados ao resultado.


In [12]:
# 4.3. Salvar o resultado final
print("Salvando Poligonos de Tempo de viagem (10,20 e 30min) final em 'isocronas_ae2sfca_102030min.geojson'...")
gdf_aneis_final.to_file(
    "./dados/overturemaps/isocrona/isocronas_ae2sfca_102030min.geojson", 
    driver="GeoJSON"
)

print("\n--- SUCESSO! ---")

Salvando Poligonos de Tempo de viagem (10,20 e 30min) final em 'isocronas_ae2sfca_102030min.geojson'...

--- SUCESSO! ---


# 5 Cálculo das Isocronas: 0-5,5-10 e 10-15min

In [13]:
# --- 3. PROCESSAMENTO EM LOTE ---

# Lista para armazenar nossos resultados (geometrias e atributos)
aneis_de_isocronas = []

print(f"\nIniciando geração de isócronas para as {len(ubs_gdf)} unidades...")
# tqdm() envolve o iterador para criar uma barra de progresso
for ubs in tqdm(ubs_gdf.itertuples(), total=len(ubs_gdf)):
    
    # Pegar o ponto de origem (geometria) da UBS atual
    ponto_origem = ubs.geometry
    
    # Pegar um ID único da UBS. 
    # Estou usando o Index (ID da linha), mas você pode usar 
    # ubs.cnes, ubs.nome_unidade, ou qualquer coluna única.
    id_origem = ubs.cnes
    
    try:
        # --- 3.1. Calcular as 3 isócronas cumulativas ---
        poly_5_gdf = c2g_utils.create_isochrone(
            graph=edges_gdf,
            center_point=ponto_origem,
            distance=5, 
            edge_attr='travel_time_min'
        )
        
        poly_10_gdf = c2g_utils.create_isochrone(
            graph=edges_gdf,
            center_point=ponto_origem,
            distance=10, 
            edge_attr='travel_time_min'
        )
        
        poly_15_gdf = c2g_utils.create_isochrone(
            graph=edges_gdf,
            center_point=ponto_origem,
            distance=15, 
            edge_attr='travel_time_min'
        )

        # --- 3.2. Extrair geometrias e tratar falhas ---
        # Se a isócrona não puder ser gerada (ex: ponto isolado),
        # a função create_isochrone retorna um GDF vazio.
        
        if not poly_5_gdf.empty:
            poly_5_cumulativo = poly_5_gdf.geometry.iloc[0]
        else:
            # Pula esta UBS se não conseguir gerar a isócrona de 5 min
            print(f"Aviso: Não foi possível gerar isócrona de 10min para ID {id_origem}. Pulando.")
            continue
            
        if not poly_10_gdf.empty:
            poly_10_cumulativo = poly_10_gdf.geometry.iloc[0]
        else:
            poly_10_cumulativo = poly_5_cumulativo # Evita erro no .difference

        if not poly_15_gdf.empty:
            poly_15_cumulativo = poly_15_gdf.geometry.iloc[0]
        else:
            poly_15_cumulativo = poly_10_cumulativo # Evita erro no .difference

        # --- 3.3. Gerar as zonas "Donut" (Anéis Discretos) ---
        zona_0_5_min = poly_5_cumulativo
        zona_5_10_min = poly_10_cumulativo.difference(poly_5_cumulativo)
        zona_10_15_min = poly_15_cumulativo.difference(poly_10_cumulativo)

        # --- 3.4. Adicionar os resultados à lista ---
        # Adicionamos um dicionário para cada anel
        
        aneis_de_isocronas.append({
            'id_origem': id_origem,
            'zona_tempo': '0-5 min',
            'geometry': zona_0_5_min
        })
        
        aneis_de_isocronas.append({
            'id_origem': id_origem,
            'zona_tempo': '5-10 min',
            'geometry': zona_5_10_min
        })
        
        aneis_de_isocronas.append({
            'id_origem': id_origem,
            'zona_tempo': '10-15 min',
            'geometry': zona_10_15_min
        })

    except Exception as e:
        print(f"Erro inesperado ao processar ID {id_origem}: {e}")

# --- 4. FINALIZAÇÃO ---

print("\nProcessamento em lote concluído.")
print(f"Total de {len(aneis_de_isocronas)} geometrias de anel geradas.")


Iniciando geração de isócronas para as 75 unidades...


100%|██████████| 75/75 [06:32<00:00,  5.23s/it]


Processamento em lote concluído.
Total de 225 geometrias de anel geradas.


In [14]:
# 4.1. Converter a lista de resultados em um GeoDataFrame
print("Convertendo resultados para GeoDataFrame...")
gdf_aneis_final = gpd.GeoDataFrame(
    aneis_de_isocronas,
    crs=CRS_PROJETADO
)

# 4.2. (Opcional) Juntar atributos das UBS (ex: nome)
# Se você usou o Index como 'id_origem', podemos facilmente
# juntar os dados originais das UBS.
if 'id_origem' in gdf_aneis_final.columns:
    # Selecionar colunas de atributos (exceto geometria)
    atributos_ubs = ubs_gdf.drop(columns='geometry') 
    
    gdf_aneis_final = gdf_aneis_final.merge(
        atributos_ubs,
        left_on='id_origem',
        right_on='cnes',
        how='left'
    )
    print("Atributos das UBS (nome, cnes, etc.) juntados ao resultado.")

Convertendo resultados para GeoDataFrame...
Atributos das UBS (nome, cnes, etc.) juntados ao resultado.


In [15]:
# 4.3. Salvar o resultado final
print("Salvando Poligonos de Tempo de viagem (5, 10 e 15min) final em 'isocronas_ae2sfca_051015min.geojson'...")
gdf_aneis_final.to_file(
    "./dados/overturemaps/isocrona/isocronas_ae2sfca_051015min.geojson", 
    driver="GeoJSON"
)

print("\n--- SUCESSO! ---")

Salvando Poligonos de Tempo de viagem (5, 10 e 15min) final em 'isocronas_ae2sfca_051015min.geojson'...

--- SUCESSO! ---
